<a href="https://colab.research.google.com/github/s1scottd/ColabStorage/blob/main/Personal_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

**Read in the mnist dataset.**

In [63]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape:  {x_test.shape}")
print(f"y_test shape:  {y_test.shape}")

x_train shape: (60000, 28, 28)
y_train shape: (60000,)
x_test shape:  (10000, 28, 28)
y_test shape:  (10000,)


**Reshape the data to (number of samples, 28, 28, 1).  Normalize the data as floating point.**

In [64]:
def reshape_and_nornmalize(data):
  shape = data.shape
  data = np.reshape(data,(shape[0],28,28,1))
  data = data/255.
  return data

In [65]:
x_train = reshape_and_nornmalize(x_train)
x_test = reshape_and_nornmalize(x_test) 
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape:  {x_test.shape}")

x_train shape: (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)


**Create the model and compile it.**

In [66]:
def convolutional_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation = "relu"),
      tf.keras.layers.Dense(10, activation="softmax")
  ])

  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  
  return model

In [67]:
model = convolutional_model()

print(f"\nSummary of the model")
model.summary()


Summary of the model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 5408)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               692352    
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
__________________________

**Create a callback that will stop fitting the model once it reaches 99.5% accuracy.**

In [68]:
class my_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=[]):
    if (logs.get("accuracy") >= 0.995):
      self.model.stop_training = True
      print(f"\nReached 99.5% accuracy so cancelling training after {epoch} epochs.")

**Fit the model.**

In [69]:
print(f"\nFit Model")

callbacks = my_callback()

history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks], verbose=2)


Fit Model
Epoch 1/10
1875/1875 - 6s - loss: 0.1487 - accuracy: 0.9562 - 6s/epoch - 3ms/step
Epoch 2/10
1875/1875 - 5s - loss: 0.0518 - accuracy: 0.9845 - 5s/epoch - 2ms/step
Epoch 3/10
1875/1875 - 4s - loss: 0.0324 - accuracy: 0.9897 - 4s/epoch - 2ms/step
Epoch 4/10
1875/1875 - 6s - loss: 0.0219 - accuracy: 0.9930 - 6s/epoch - 3ms/step
Epoch 5/10

Reached 99.5% accuracy so cancelling training after 4 epochs.
1875/1875 - 4s - loss: 0.0138 - accuracy: 0.9956 - 4s/epoch - 2ms/step


**Evaluate the model**

In [70]:
# Evaluate on the test set
print(f'\nEvaluate Model:')
test_loss = model.evaluate(x_test, y_test)


Evaluate Model:
313/313 [==============================] - 1s 2ms/step - loss: 0.0479 - accuracy: 0.9852
